## Hey There!
Modeling Approach: Long Short Term Memory Neural Network <br>
@Spencer your code is in the 1D_CNN File

# Set up

In [ ]:
# @title Imports 
import pandas as pd
import numpy as np

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.metrics import AUC
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import random
import os
import json



import matplotlib.pyplot as plt
from IPython.display import clear_output 

pd.set_option('display.max_columns', None)

Connect to TPU

In [ ]:
## Connect to TPU ##

# TPU Configuration
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

with tf.device('/TPU:0'):
  c = tf.matmul(a, b)
print("c device: ", c.device)
print(c)
strategy = tf.distribute.TPUStrategy(resolver)

# Clear ouput 
clear_output()

In [ ]:
# Connect to Drive 
from google.colab import drive
drive.mount('/content/drive')
clear_output()

#Data Preprocessing / Normalization 

Get and Preprocess X

We should come up with a better long-term solution for the issue of not being able to load data easily :|


In [ ]:
file_location_spencer: str = '/content/drive/MyDrive/DS4H/Data/wideX_Smoothed.csv'
file_location_franklin: str = '/content/drive/MyDrive/Colab Notebooks/Sleep Disorder Project/Project Code + Data/Minute-by-Minute Data/Wide Data [Smoothed]/wideX_Smoothed.csv'

In [ ]:
#Load data
Wide_X = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Sleep Disorder Project/Project Code + Data/Minute-by-Minute Data/Wide Data [Smoothed]/wideX_Smoothed.csv')
Wide_X = Wide_X.drop("Unnamed: 0", axis = 1)

#Wide_X.fillna(0)
Wide_X = Wide_X.where(Wide_X > 0.0000001, 0)

In [ ]:
Wide_X

In [ ]:
# Seperate SEQN from dataframe
SEQN = Wide_X.pop("SEQN")
data_wide = Wide_X

# Normalize from 0.01 to 1
scaler = MinMaxScaler()
scaler.fit(data_wide)
data_wide = scaler.transform(data_wide)

# Convert DF to array
data_wide = np.array(data_wide)

# Set 3-D shape
data_wide = data_wide.reshape(len(data_wide),len(data_wide[0]),1)
data_wide = data_wide.astype(np.float32)
# Set shape for model
data_wide_shape = (data_wide.shape[1],data_wide.shape[2])



print("Shape:", data_wide.shape)

Preprocess y

In [ ]:
# Load Data Y ----
Y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Sleep Disorder Project/Project Code + Data/Minute-by-Minute Data/Wide Data [Smoothed]/y.csv')
Y = Y.drop("Unnamed: 0", axis = 1)
#Make y array as well 
y = np.hstack(np.asarray(Y)).reshape(len(Y),1)
print(y.shape)
y = y.astype(np.float32)

### Stratified Train Test Split

In [ ]:
# Train Test Split
X_train, X_test, y_train, y_test, ids_train, ids_test = train_test_split(data_wide, y, SEQN, test_size=0.2, stratify = y, random_state = 41)

# Train Validation Split
X_train, X_val, y_train, y_val, ids_train, ids_val = train_test_split(X_train, y_train, ids_train, test_size=0.2, stratify = y_train, random_state = 32)

# Modeling!

Make Model

In [ ]:
#Make Model 
def create_model():
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.LSTM(
            units=400, # adjust size 
          input_shape=(10080, 1)
          
      )
  )
  model.add(Dense(100, activation='relu'))
  model.add(tf.keras.layers.Dropout(rate=0.2))
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
  return model

Compile Model 

In [ ]:
from keras import optimizers

# Compile Model 
with strategy.scope():
  model = create_model()
  model.compile(
  loss = tf.keras.losses.BinaryCrossentropy(from_logits=False),
  optimizer= tf.keras.optimizers.Adam(),
  metrics= tf.keras.metrics.AUC(name="auc")
)

In [ ]:
# Save Model 
model.save_weights("reset_weights.h5")

def reset_model():
  model.load_weights("reset_weights.h5")

## Model Visuals

In [ ]:
# Model Summary
model.summary()

In [ ]:
# Plot Model Architecture 
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

##Train Model 

In [ ]:
## Parameters##

# Save Best Weights 
filepath = "VAL-AUC-impr(fr).h5"

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath, #=checkpoint_filepath,
    verbose = 1,
    monitor='val_auc',
    mode='max',
    save_best_only=True)


# Balance Weights 
n_sd = sum(y)
n_no_sd = len(y) - n_sd
class_weights = {0: (n_no_sd/len(y)),                # Custom Setting Class Weights 
                1: (n_sd/len(y))}

In [ ]:
# Fit model                                              
tf.keras.backend.clear_session()
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=500, 
    batch_size= 32, # Change size 
    shuffle=False,
    verbose = 1,
    class_weight=class_weights,
    callbacks=[model_checkpoint_callback]
)

## Graphing

Evaulate 

In [ ]:
history_df = pd.DataFrame({'auc':history.history['auc'], 'val_auc':history.history['val_auc']})
history_df.to_csv("history_df.csv", index = False)

# Plot history: AUC
plt.plot(history_df['auc'], label='Training')
plt.plot(history_df['val_auc'], label='Validation')
plt.title('AUC')
plt.ylabel('AUC')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()
plt.savefig('')

In [ ]:
# Evaluate 
model.evaluate(X_test, y_test, batch_size = 32)
#Keras Test AUC
from keras.models import load_model
test_set_model = keras.models.load_model('VAL-AUC-impr(fr).h5')
results = test_set_model.evaluate(X_test, y_test, batch_size = 32)
print("--------------------------")
print(f'Keras Model Loading Test AUC: {round(results[1],2)}')
print("--------------------------")


#SKLearn Test AUC 
preds = test_set_model.predict(X_test)
from sklearn.metrics import r2_score, roc_auc_score
print(f'SKLearn Test AUC: {round(roc_auc_score(y_test, preds),3)}')
print("--------------------------")
plt.plot(preds)
plt.title('Predictions for Test Set')
plt.show()
plt.scatter(y_test, preds)

#K-Fold Cross Validation

In [ ]:
%%time
from sklearn.model_selection import StratifiedKFold

# Global Score List Buckets
cv_test_scores=[]
cv_val_scores=[]

# K fold parameters 
seed = 7
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

# run K-fold 
count = 1
for train, val in kfold.split(X_train, y_train):
  
  # Create New Training Set 
  X_training = X_train[train]
  y_training = y_train[train]
  # Create new Validation Sets 
  X_val = X_train[val]
  y_val = y_train[val]

  # Reset model 
  print("Resetting model..")
  reset_model()
  # Checkpoints
  filepath = "VAL-AUC-impr(fr)" + str(count) +".h5"

  model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath, #=checkpoint_filepath,
    verbose = 2,
    monitor='val_auc',
    mode='max',
    save_best_only=True)

  # fit model   
  tf.keras.backend.clear_session()
  history = model.fit(
      X_training, y_training,
      validation_data=(X_val, y_val),
      epochs=30, # Make Epochs larger
      batch_size= 32, # Change size 
      shuffle=False,
      verbose = 1,
      class_weight=class_weights,
      callbacks=[model_checkpoint_callback]
  )


 
  # Eval model 
  # model_eval()
  from keras.models import load_model
  test_set_model = keras.models.load_model(filepath)
  print(" \n-\n-\n-\n-\n-\n-\n-\n-\n-\ntest set:")
  scores = test_set_model.evaluate(X_test, y_test, batch_size=64) # Test Set
  cv_test_scores.append(scores[1])
  print(" \n-\n-\n-\n-\n-\n-\n-\n-\n-\nval set:")
  scores = test_set_model.evaluate(X_val, y_val, batch_size=64)  # Validation Set
  cv_val_scores.append(scores[1])



  # Save model 
  print("Saving model...")
  #save_model(count)
  # Save files
  print("Saving files...")
  #save_files(count)

  # increment 
  print("FINISHED CYCLE NUMBER:", count)
  count += 1 

# Score Eval
print("\nCV Test AUC----------------------------")
print("Individual scores:", cv_test_scores)
print("Mean:", np.mean(cv_test_scores))
print("std:", np.std(cv_test_scores))
print("\nCV Val AUC-----------------------------")
print("Individual scores:", cv_val_scores)
print("Mean:", np.mean(cv_val_scores))
print("std:", np.std(cv_val_scores))

# Spencer Implementation

In [ ]:
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
#Load data
file_location_spencer: str = '/content/drive/MyDrive/DS4H/Data/wideX_Smoothed.csv'
Wide_X = pd.read_csv(file_location_spencer)
Wide_X = Wide_X.drop("Unnamed: 0", axis = 1)
Wide_X = Wide_X.fillna(0)
Wide_X = round(Wide_X, 6)
Wide_X.head()

In [ ]:
#Load response variable
file_location_spencer_y: str = '/content/drive/MyDrive/DS4H/Data/data_all_features_hourly.csv'
df_demographic = pd.read_csv(file_location_spencer_y)
df_demographic = df_demographic.drop("Unnamed: 0", axis = 1)
df_demographic = df_demographic.fillna(0)
df_demographic = round(df_demographic, 6)
df_demographic.head()

## Choose the response variable from the dataframe above - can be anything numeric

In [ ]:
response = 'sleep_disorder'

In [ ]:
df_response = df_demographic[['patient_id', response]]
df_response

In [ ]:
def transform_y(y_df: pd.DataFrame) -> np.array:
  """
  Used for cleaning
  """
  y = np.hstack(np.asarray(y_df)).reshape(len(y_df), 1)
  print(y.shape)
  y = y.astype(np.int32)

  return y

In [ ]:
def scale_data(data_wide: pd.DataFrame) -> np.array:
  """
  Used for cleaning
  """
  # Normalize from 0.01 to 1
  scaler = MinMaxScaler()
  scaler.fit(data_wide)
  data_wide = scaler.transform(data_wide)

  # Convert DF to array
  data_wide = np.array(data_wide)

  # Set 3-D shape
  data_wide = data_wide.reshape(len(data_wide),len(data_wide[0]),1)
  data_wide = data_wide.astype(np.float32)
  # Set shape for model
  data_wide_shape = (data_wide.shape[1],data_wide.shape[2])

  print("Shape:", data_wide.shape)

  return data_wide

In [ ]:
def generate_data_set(df_minutes: pd.DataFrame, df_demographic: pd.DataFrame):
  """
  This funciton takes the minute-level activity data and joins it onto a response variable, then creates the X_train, y_train, X_test, and y_test arrays

  """

  # step 1: join the dataframes to get the response variable 
  df_joined = df_minutes.merge(df_demographic, left_on='SEQN', right_on='patient_id', how='inner')
  df_joined = df_joined.drop_duplicates()

  # remove the ID numbers 
  if "SEQN" in df_joined.columns: 
    SEQN = df_joined.pop("SEQN")
  if "patient_id" in df_joined.columns: 
    patient_id = df_joined.pop("patient_id")

  # Seperate y from dataframe
  y = df_joined.pop(response)

  # define X
  X = df_joined.copy()

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=15)

  # scale the feature set 
  X_train = scale_data(data_wide=X_train)
  X_test = scale_data(data_wide=X_test)

  # transform the response variable and reshape
  y_train = transform_y(y_train)
  y_test = transform_y(y_test)

  return {'X_train': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test': y_test}  

data_dict: dict = generate_data_set(df_minutes=Wide_X, df_demographic=df_response)

X_train: np.array = data_dict['X_train']
X_test: np.array = data_dict['X_test']
y_train: np.array = data_dict['y_train']
y_test: np.array = data_dict['y_test']

print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

In [ ]:
def generate_cm(y_test, y_pred):
  """
  generate confusion matrix
  """
  cm = confusion_matrix(y_test, y_pred)
  disp = ConfusionMatrixDisplay(confusion_matrix=cm)
  disp.plot()
  plt.show()

In [ ]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
  verbose, epochs, batch_size = 1, 5, 64
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  model = Sequential()
  model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
  model.add(Dropout(0.5))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(n_outputs, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	
  # fit network
  model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
  
  y_pred = model.predict(testX)

  # evaluate model
  _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    
  generate_cm(y_test, y_pred)

  return accuracy
 
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))


In [ ]:
score = evaluate_model(X_train, y_train, X_test, y_test)
score = score * 100.0
print(score)

In [ ]:
# # run an experiment
# def run_experiment(repeats=10):
# 	# load data

#   # TODO add stochasticity back in here vvv
# 	trainX, trainy, testX, testy = X_train, y_train, X_test, y_test
# 	# repeat experiment
# 	scores = list()
# 	for r in range(repeats):
# 		score = evaluate_model(trainX, trainy, testX, testy)
# 		score = score * 100.0
# 		print('>#%d: %.3f' % (r+1, score))
# 		scores.append(score)
# 	# summarize results
# 	summarize_results(scores)
 
# # run the experiment
# run_experiment()  # <-- uncomment for experiment run 